## Load packages

In [1]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import glob
import json
import pickle as p
from math import nan
from mapboxgl.viz import *
from mapboxgl.utils import *
from keplergl import KeplerGl
# mapbox://styles/mapbox/satellite-streets-v11
from sklearn.isotonic import IsotonicRegression
import datetime
import matplotlib.pyplot as plt
import matplotlib
import statsmodels.api as sm
import os
import csv

In [2]:
plt.rcParams["figure.figsize"] = (30,16)
plt.rcParams.update({'font.size': 32})
plt.rcParams.update({'figure.autolayout': True})


import warnings
warnings.filterwarnings('ignore')

## Load data

In [3]:
path = "C:/Users/Hamidreza/Dropbox/UAS4T/sample_dataset/" 
path = "C:/Users/Hamidreza/Dropbox/UAS4T/competition_dataset/"
# load samaple data
meta_dfs = []
trace_dfs = []

df_street = pd.read_csv('street_information.csv',header=0)

with open(path+'pneuma_competition_dataset.p', 'rb') as f:
    meta_df, trace_df = p.load(f)
trace_df.reset_index(inplace=True)

trace_df['time'] = pd.to_datetime(trace_df['time'], unit = 's', origin=pd.Timestamp('2020-08-04T08'))
phi = trace_df['lat'].mean()
# transfomration are from https://en.wikipedia.org/wiki/Geographic_coordinate_system
lat_to_y = 111132.92 - 559.82 * np.cos(2*phi) + 1.175 * np.cos(4*phi) - 0.0023 * np.cos(6*phi)
lon_to_x = 111412.84 * np.cos(phi) - 93.5 * np.cos(3*phi) + 0.118 * np.cos(5*phi)
trace_df['X'] = trace_df['lon'] * lon_to_x
trace_df['Y'] = trace_df['lat'] * lat_to_y
df_street['X_1'] = df_street['lon_1'] * lon_to_x
df_street['Y_1'] = df_street['lat_1'] * lat_to_y
df_street['X_2'] = df_street['lon_2'] * lon_to_x
df_street['Y_2'] = df_street['lat_2'] * lat_to_y
# set the bottom lett corner to (0,0)
df_street['X_1'] = df_street['X_1']-trace_df['X'].min()
df_street['Y_1'] = df_street['Y_1']-trace_df['Y'].min()
df_street['X_2'] = df_street['X_2']-trace_df['X'].min()
df_street['Y_2'] = df_street['Y_2']-trace_df['Y'].min()
df_street['heading'] = np.arctan2(df_street['Y_2']-df_street['Y_1'],df_street['X_2']-df_street['X_1'])
trace_df['X'] = trace_df['X']-trace_df['X'].min()
trace_df['Y'] = trace_df['Y']-trace_df['Y'].min()

## Inititialize road data

In [4]:
# determine offet for each road
df_street['offset'] = 0
df_street['length'] = 0
# determine the rotated coordinates
for i in range(len(df_street)):
    # retrieve related street names
    street_name = df_street.loc[i]['Street']
    rot = -df_street.loc[i]['heading']
    R = np.array([[np.cos(rot), -np.sin(rot)],[np.sin(rot), np.cos(rot)]])
    # determine the new coordinates in the rotated framework
    df_street.loc[i,'X_1_r'] = df_street.loc[i,'X_1']
    df_street.loc[i,'Y_1_r'] = df_street.loc[i,'Y_1']
    df_street.loc[i,'X_2_r'] = df_street.loc[i,'X_2']
    df_street.loc[i,'Y_2_r'] = df_street.loc[i,'Y_2']
    df_street.loc[i,['X_1_r','Y_1_r']] = np.dot(R.squeeze(),df_street.loc[i,['X_1','Y_1']].as_matrix().T).T
    df_street.loc[i,['X_2_r','Y_2_r']] = np.dot(R.squeeze(),df_street.loc[i,['X_2','Y_2']].as_matrix().T).T
    df_street.loc[i,'length'] = df_street.loc[i,'X_2_r'] -  df_street.loc[i,'X_1_r']


# Oktovriou:
# set the point reference point to (117,93, 260.50) (loc 4)
df_street.loc[4,'offset'] = 0
df_street.loc[5,'offset'] = df_street.loc[4,'length']
df_street.loc[6,'offset'] = df_street.loc[5,'offset'] + df_street.loc[5,'length']
df_street.loc[7,'offset'] = df_street.loc[6,'offset'] + df_street.loc[6,'length']
df_street.loc[8,'offset'] = df_street.loc[7,'offset'] + df_street.loc[7,'length']
df_street.loc[3,'offset'] = - df_street.loc[3,'length']
df_street.loc[2,'offset'] = df_street.loc[3,'offset'] - df_street.loc[2,'length']
df_street.loc[1,'offset'] = df_street.loc[2,'offset'] - df_street.loc[1,'length']
# Metsovou
df_street.loc[10,'offset'] = df_street.loc[9,'length']
# Louliaanou_W
df_street.loc[11,'offset'] = - df_street.loc[11,'length']
# Louliaanou_E
df_street.loc[12,'offset'] = 0
# Vasileous
df_street.loc[12,'offset'] = 0
# Deligianni
df_street.loc[13,'offset'] = df_street.loc[12,'length']+7
# Cheiden_E
df_street.loc[18,'offset'] = 0
# Cheiden_W
df_street.loc[19,'offset'] =  - df_street.loc[19,'length']
# Kotsika
df_street.loc[20,'offset'] = 0
# Egiptou
df_street.loc[21,'offset'] = 0
df_street.loc[22,'offset'] = df_street.loc[21,'length'] + 5
# Mavrommateon
df_street.loc[24,'offset'] = - df_street.loc[24,'length']
df_street.loc[23,'offset'] = - df_street.loc[23,'length'] + df_street.loc[24,'offset']
df_street.loc[25,'offset'] = 0
df_street.loc[26,'offset'] = df_street.loc[25,'length']
df_street.loc[27,'offset'] = df_street.loc[26,'length'] + df_street.loc[26,'offset']
# Rethimnou
df_street.loc[28,'offset'] = - df_street.loc[28,'length']
# Saripolou (does not go completely up to Alexandrtas, so used length Rethimnou for offset)
df_street.loc[29,'offset'] = - df_street.loc[28,'length']
# Mpoumpoulinas
df_street.loc[30,'offset'] = - df_street.loc[30,'length']
# Ziami
df_street.loc[32,'offset'] = - df_street.loc[32,'length']
df_street.loc[31,'offset'] = - df_street.loc[31,'length'] + df_street.loc[32,'offset']
# Navarchou
df_street.loc[34,'offset'] = - df_street.loc[34,'length']
df_street.loc[33,'offset'] = - df_street.loc[33,'length'] + df_street.loc[34,'offset']
# Spirou
df_street.loc[36,'offset'] = - df_street.loc[36,'length']
df_street.loc[35,'offset'] = - df_street.loc[35,'length'] + df_street.loc[36,'offset']
# Kallidromiou
df_street.loc[37,'offset'] = - df_street.loc[37,'length']
# Enianois
df_street.loc[38,'offset'] = - df_street.loc[38,'length']
# Feron
df_street.loc[39,'offset'] = - df_street.loc[39,'length']
# Gkiilfordou
df_street.loc[40,'offset'] = - df_street.loc[40,'length']
# Gkiilfordou
df_street.loc[40,'offset'] = - df_street.loc[40,'length']
# Skaramagka
df_street.loc[41,'offset'] = - df_street.loc[41,'length']
df_street

,Street,lat_1,lon_1,lat_2,lon_2,Unnamed: 5,offset,dir,segment,X_1,Y_1,X_2,Y_2,heading,length,X_1_r,Y_1_r,X_2_r,Y_2_r
0,Alexandras,37.991767,23.731317,37.990439,23.739712,NaN,0.000000,WE,0,110.787703,248.335608,1005.939543,101.370507,-0.162727,907.135910,69.091243,263.003592,976.227153,263.003592
1,Oktovriou,37.990095,23.730865,37.991268,23.731160,NaN,-204.359169,SN,0,62.591320,63.301233,94.046924,193.113029,1.333062,133.568549,76.261175,-45.923334,209.829724,-45.923334
2,Oktovriou,37.991268,23.731160,37.991638,23.731253,NaN,-70.790621,SN,1,94.046924,193.113029,103.963436,234.059631,1.333190,42.130291,209.823848,-45.950174,251.954139,-45.950174
3,Oktovriou,37.991638,23.731253,37.991878,23.731354,NaN,-28.660330,SN,2,103.963436,234.059631,114.732982,260.619589,1.185574,28.660330,255.972387,-8.393086,284.632716,-8.393086
4,Oktovriou,37.991878,23.731354,37.992030,23.731415,LT,0.000000,SN,3,114.732982,260.619589,121.237361,277.440896,1.201830,18.035058,284.458687,-13.018538,302.493745,-13.018538
5,Oktovriou,37.992030,23.731415,37.992189,23.731427,NaN,18.035058,SN,4,121.237361,277.440896,122.516911,295.036868,1.498206,17.642434,285.503207,-100.796166,303.145641,-100.796166
6,Oktovriou,37.992189,23.731427,37.992436,23.731486,NaN,35.677492,SN,5,122.516911,295.036868,128.808031,322.371491,1.344583,28.049240,314.999284,-53.222158,343.048523,-53.222158
7,Oktovriou,37.992436,23.731486,37.992890,23.731599,NaN,63.726732,SN,6,128.808031,322.371491,140.857127,372.614079,1.335423,51.667188,343.521660,-50.077505,395.188848,-50.077505
8,Oktovriou,37.992890,23.731599,37.993689,23.731799,NaN,115.393920,SN,7,140.857127,372.614079,162.182960,461.036606,1.334135,90.957872,395.253026,-49.568415,486.210898,-49.568415
9,Metsovou,37.990815,23.731174,37.990567,23.732776,NaN,0.000000,WE,0,95.539732,142.981108,266.359654,115.535818,-0.159306,173.010664,71.648367,156.326418,244.659031,156.326418


## Calculate heading

In [5]:
d = 12
min_speed = 2 
trace_df['heading_raw'] = np.arctan2(trace_df['Y'].diff(d),trace_df['X'].diff(d))#/np.pi*180
#trace_df['heading_raw'] += 360 
trace_df['heading'] = np.nan
# if the car does not move set heading to NaN
trace_df.loc[(trace_df['Y'].diff(d)==0)&(trace_df['X'].diff(d)==0),'heading_raw'] = np.nan
trace_df.loc[(trace_df['speed']<=min_speed)|(trace_df.shift(d)['speed']<=min_speed),'heading_raw'] = np.nan
trace_df.loc[(trace_df['track_id']!=trace_df['track_id'].shift(d)),'heading_raw'] = np.nan

grouped = trace_df.groupby('track_id')
for name, group in grouped:
    if name%10 == 0:
        print(name)
    # moving average for heading with size 10 (0.4 second)
    group['heading'] = np.arctan2(group['heading_raw'].apply(np.sin).rolling(12, win_type = 'triang', min_periods = 6).mean(),group['heading_raw'].apply(np.cos).rolling(12, win_type = 'triang', min_periods = 6).mean())
    # fill NaN backward/forward 
    group['heading'] = group['heading'].fillna(method = 'bfill')
    group['heading'] = group['heading'].fillna(method = 'ffill')
    trace_df.loc[group.index,'heading'] = group['heading']

    
    
# taking care of trips with small number of samples (no heading can be determined via above procedure)
no_heading = trace_df[['track_id','heading']].groupby('track_id').mean().isna()
no_heading = no_heading[no_heading['heading']].index.values

d = 4
min_speed = 0.1 

grouped = trace_df.groupby('track_id')
for name, group in grouped:
    if name not in  no_heading:
        continue
    # moving average for heading with size 10 (0.4 second)
    group['heading_raw'] = np.arctan2(group['Y'].diff(d),group['X'].diff(d))#/np.pi*180
    group['heading'] = np.arctan2(group['heading_raw'].apply(np.sin).rolling(4, win_type = 'triang', min_periods = 2).mean(),group['heading_raw'].apply(np.cos).rolling(12, win_type = 'triang', min_periods = 6).mean())
    # fill NaN backward/forward 
    group['heading'] = group['heading'].fillna(method = 'bfill')
    group['heading'] = group['heading'].fillna(method = 'ffill')
    trace_df.loc[group.index,'heading'] = group['heading']
    trace_df.loc[group.index,'heading_raw'] = group['heading_raw']
    
# taking care of trips with small number of samples (no heading can be determined via above procedure)
no_heading = trace_df[['track_id','heading']].groupby('track_id').mean().isna()
no_heading = no_heading[no_heading['heading']].index.values

d = 2
min_speed = 0.1 

grouped = trace_df.groupby('track_id')
for name, group in grouped:
    if name not in  no_heading:
        continue
    # moving average for heading with size 10 (0.4 second)
    group['heading'] = np.arctan2(group['Y'].diff(d),group['X'].diff(d))#/np.pi*180
    # fill NaN backward/forward 
    group['heading'] = group['heading'].fillna(method = 'bfill')
    group['heading'] = group['heading'].fillna(method = 'ffill')
    trace_df.loc[group.index,'heading'] = group['heading']

10.0
20.0
30.0
40.0
50.0
60.0
70.0
80.0
90.0
100.0
110.0
120.0
130.0
140.0
150.0
160.0
170.0
180.0
190.0
200.0
210.0
220.0
230.0
240.0
250.0
260.0
270.0
280.0
290.0
300.0
310.0
320.0
330.0
340.0
350.0
360.0
370.0
380.0
390.0
400.0
410.0
420.0
430.0
440.0
450.0
460.0
470.0
480.0
490.0
500.0
510.0
520.0
530.0
540.0
550.0
560.0
570.0
580.0
590.0
600.0
610.0
620.0
630.0
640.0
650.0
660.0
670.0
680.0
690.0
700.0
710.0
720.0
730.0
740.0
750.0
760.0
770.0
780.0
790.0
800.0
810.0
820.0
830.0
840.0
850.0
860.0
870.0
880.0
890.0
900.0
910.0
920.0
930.0
940.0
950.0
960.0
970.0
980.0
990.0
1000.0
1010.0
1020.0
1030.0
1040.0
1050.0
1060.0
1070.0
1080.0
1090.0
1100.0
1110.0
1120.0
1130.0
1140.0
1150.0
1160.0
1170.0
1180.0
1190.0
1200.0
1210.0
1220.0
1230.0
1240.0
1250.0
1260.0
1270.0
1280.0
1290.0
1300.0
1310.0
1320.0
1330.0
1340.0
1350.0
1360.0
1370.0
1380.0
1390.0
1400.0
1410.0
1420.0
1430.0
1440.0
1450.0
1460.0
1470.0
1480.0
1490.0
1500.0
1510.0
1520.0
1530.0
1540.0
1550.0
1560.0
1570.0
1580.0
15

In [6]:
trace_df_full = trace_df.copy()

In [7]:
trip_set = np.arange(0,2000,40)
trace_df = trace_df_full[trace_df_full['track_id'].isin(trip_set)]
trace_df = trace_df_full.copy()
#trace_df = trace_df.iloc[::100,:]    

## Road assignment

In [8]:
deg_tol = 15 * (np.pi/180)
deg_coef = 100 * (180/np.pi)
Y_tol = 15
Y_coef = 1000
X_tol = 5
X_coef = 10000

trace_df['score'] = deg_coef * 10 + Y_coef * 20 + X_coef * 5
trace_df['road'] = None
trace_df['dir'] = None
trace_df['X_road'] = None
trace_df['Y_road'] = None
trace_df['heading_road'] = None

for i in range(len(df_street)):
    # retrieve related street names
    street_name = df_street.loc[i]['Street']
    rot = -df_street.loc[i]['heading']
#    if (street_name == 'Out_ramp_1')|(street_name == 'Out_ramp_2')|(street_name == 'In_ramp'):
#        continue
#    if (street_name!= 'Mpoumpoulinas')&(street_name!= 'Loulianou_E'):
#        continue
    print(i,street_name)
    
    # rotation matrix
    R = np.array([[np.cos(rot), -np.sin(rot)],[np.sin(rot), np.cos(rot)]])
    # determine the new coordinates in the rotated framework
    df_street.loc[i,'X_1_r'] = df_street.loc[i,'X_1']
    df_street.loc[i,'Y_1_r'] = df_street.loc[i,'Y_1']
    df_street.loc[i,'X_2_r'] = df_street.loc[i,'X_2']
    df_street.loc[i,'Y_2_r'] = df_street.loc[i,'Y_2']
    df_street.loc[i,['X_1_r','Y_1_r']] = np.dot(R.squeeze(),df_street.loc[i,['X_1','Y_1']].as_matrix().T).T
    df_street.loc[i,['X_2_r','Y_2_r']] = np.dot(R.squeeze(),df_street.loc[i,['X_2','Y_2']].as_matrix().T).T

    # GPS traces: determine the new coordinates in the rotated framework
    trace_df['X_r'] = trace_df['X']
    trace_df['Y_r'] = trace_df['Y']
    trace_df[['X_r','Y_r']] = np.dot(R.squeeze(),trace_df[['X','Y']].as_matrix().T).T
    # subtracts the reference point of the road (one of the end)
    trace_df['Y_r'] = trace_df['Y_r'] - np.mean([df_street.loc[i,'Y_1_r'],df_street.loc[i,'Y_2_r']])
    trace_df['X_r'] = trace_df['X_r'] - df_street.loc[i,'X_1_r']
    # the new heading angle (add the rotation angle)
    trace_df['heading_r'] = trace_df['heading'] + rot
    # make sure angles is in [-np.pi,np.pi]
    trace_df['heading_r'] = (trace_df['heading_r']+4*np.pi).mod(2*np.pi)
    trace_df['heading_r'] = (trace_df['heading_r']<=np.pi)*trace_df['heading_r'] + (trace_df['heading_r']>np.pi)*(trace_df['heading_r'] - 2*np.pi)
    
    # determine the score
    if street_name.find('ramp')>-1:
        trace_df['score_temp'] = np.maximum(0, trace_df['heading_r'].abs() - deg_tol) * deg_coef 
    else:
        trace_df['score_temp'] = np.minimum(np.maximum(0, trace_df['heading_r'].abs() - deg_tol),np.maximum(0, np.pi-trace_df['heading_r'].abs() - deg_tol)) * deg_coef 
    trace_df['score_temp'] += np.maximum(0,trace_df['Y_r'].abs() - Y_tol) * Y_coef 
    trace_df['score_temp'] += np.maximum((np.maximum(0,-trace_df['X_r']-X_tol)),np.maximum(0,trace_df['X_r']-df_street.loc[i,'X_2_r']+df_street.loc[i,'X_1_r']-X_tol)) * X_coef
    selected_trips = trace_df['score_temp']<trace_df['score']
    
    # update name & score of the newly found closest road
    trace_df.loc[selected_trips,'road'] = street_name
    trace_df.loc[selected_trips,'score'] = trace_df.loc[selected_trips,'score_temp']

    trace_df.loc[selected_trips,'X_road'] = trace_df.loc[selected_trips,'X_r'] +  df_street.loc[i,'offset']
    trace_df.loc[selected_trips,'Y_road'] = trace_df.loc[selected_trips,'Y_r']
    trace_df.loc[selected_trips,'heading_road'] = trace_df.loc[selected_trips,'heading_r']

    if ((rot <= (45 * np.pi/180))&(rot > (-45 * np.pi/180))):
        trace_df.loc[selected_trips & (trace_df['heading_r'].abs()<=np.pi/2),'dir'] = 'EB'
        trace_df.loc[selected_trips & (trace_df['heading_r'].abs()>np.pi/2),'dir'] = 'WB'
    if ((rot < (-135 * np.pi/180)) | (rot > (135 * np.pi/180))):
        trace_df.loc[selected_trips & (trace_df['heading_r'].abs()<=np.pi/2),'dir'] = 'WB'
        trace_df.loc[selected_trips & (trace_df['heading_r'].abs()<=np.pi/2),'dir'] = 'EB'
    if ((rot > (45 * np.pi/180))&(rot <= (135 * np.pi/180))):
        trace_df.loc[selected_trips & (trace_df['heading_r'].abs()<=np.pi/2),'dir'] = 'SB'
        trace_df.loc[selected_trips & (trace_df['heading_r'].abs()>np.pi/2),'dir'] = 'NB'
    if ((rot > (-135 * np.pi/180))&(rot < (-45 * np.pi/180))):
        trace_df.loc[selected_trips & (trace_df['heading_r'].abs()<=np.pi/2),'dir'] = 'NB'
        trace_df.loc[selected_trips & (trace_df['heading_r'].abs()>np.pi/2),'dir'] = 'SB'


#trace_df.loc[(trace_df['Y_r']<20)&(trace_df['Y_r']>-20)&(trace_df['heading_r']<45)&(trace_df['heading_r']>-45),'road'] = 'E Alexandras'
#trace_df.loc[(trace_df['Y_r']<20)&(trace_df['Y_r']>-20)&((trace_df['heading_r']<-135)|(trace_df['heading_r']>135)),'road'] = 'W Alexandras'
trace_df['heading_road'] = (trace_df['heading_road']+4*np.pi).mod(2*np.pi)
trace_df['heading_road'] = (trace_df['heading_road'].abs()<=np.pi/2)*trace_df['heading_road'] + (trace_df['heading_road'].abs()>np.pi/2)*(trace_df['heading_road'] - np.pi)


0 Alexandras
1 Oktovriou
2 Oktovriou
3 Oktovriou
4 Oktovriou
5 Oktovriou
6 Oktovriou
7 Oktovriou
8 Oktovriou
9 Metsovou
10 Metsovou
11 Loulianou_W
12 Loulianou_E
13 Vasileos
14 Deligianni
15 In_ramp
16 Out_ramp_1
17 Out_ramp_2
18 Cheiden_E
19 Cheiden_W
20 Kotsika
21 Egiptou
22 Egiptou
23 Mavrommaeon_S
24 Mavrommaeon_S
25 Mavrommaeon_N
26 Mavrommaeon_N
27 Mavrommaeon_N
28 Rethimnou
29 Saripolou
30 Mpoumpoulinas
31 Zaimi
32 Zaimi
33 Navarchou
34 Navarchou
35 Spirou
36 Spirou
37 Kallidromiou
38 Enianos
39 Feron
40 Gkillfordou
41 Skaramagka


In [9]:
## MA for road column (majority vote)
#code,label = pd.factorize(trace_df['road'])
#trace_df['road_smooth'] = code
#
#grouped = trace_df.groupby('track_id')
#for name, group in grouped:
#    group['road_smooth'] = group['road_smooth'].rolling(window=25).apply(lambda x: mode(x)[0])
#    group['road_smooth'] = group['road_smooth'].fillna(method='bfill')
#    group['road_smooth'] = group['road_smooth'].fillna(method='ffill')
#    trace_df.loc[group.index,'road_smooth'] = label[group['road_smooth'].astype(int)]

In [10]:
trace_df[['track_id', 'lat', 'lon', 'speed', 'time', 'road', 'dir', 'X_road', 'Y_road', 'heading_road']].to_csv('traces_road.csv',index=False)

In [11]:
df_street.to_csv('street_information.csv',index=False)

## Visualization

In [39]:
# trip_set = np.arange(120,140)
trip_set = no_heading
sample_trip = trace_df[trace_df['track_id'].isin(trip_set)]
#sample_trip = sample_trip[['track_id','lat','lon','time','road','dir','heading_road','X_road','Y_road']]
map_2 = KeplerGl(height=800, data={'trip#1': sample_trip})
map_2

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'trip#1':          index  track_id        lat        lon    speed  lon_accel  lat_accel  \
4408…

In [52]:
 trip_set = np.arange(120,140)
sample_trip = trace_df[trace_df['track_id'].isin(trip_set)]
sample_trip = sample_trip[['track_id','lat','lon','time','road','dir','heading']]
map_3 = KeplerGl(height=800, data={'trip#1': sample_trip})
map_3

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'trip#1':          track_id        lat        lon                          time  \
45804       …

In [15]:
trace_df[['track_id', 'lat', 'lon', 'speed', 'time', 'road', 'dir', 'X_road', 'Y_road', 'heading_road']].to_csv('traces_road.csv',index=False)

In [16]:
df = trace_df[['track_id', 'lat', 'lon', 'speed', 'time', 'road', 'dir', 'X_road', 'Y_road', 'heading_road']]

,track_id,lat,lon,speed,time,road,dir,X_road,Y_road,heading_road
683,4.0,37.991066,23.735283,1.6859,2020-08-04 08:00:00.000000000,Alexandras,EB,429.873,-8.03973,2.99241
684,4.0,37.991066,23.735283,1.6795,2020-08-04 08:00:00.039999962,Alexandras,EB,429.873,-8.03973,2.99241
685,4.0,37.991066,23.735283,1.6900,2020-08-04 08:00:00.079999924,Alexandras,EB,429.873,-8.03973,2.99241
686,4.0,37.991066,23.735284,1.7135,2020-08-04 08:00:00.119999886,Alexandras,EB,429.978,-8.02245,2.99241
687,4.0,37.991066,23.735284,1.7560,2020-08-04 08:00:00.160000086,Alexandras,EB,429.978,-8.02245,2.99241
...,...,...,...,...,...,...,...,...,...,...
6322351,2296.0,37.992729,23.731518,0.9676,2020-08-04 08:13:29.039999962,Oktovriou,SB,95.2702,4.24372,-0.0565994
6322352,2296.0,37.992729,23.731518,0.9323,2020-08-04 08:13:29.079999924,Oktovriou,SB,95.2702,4.24372,-0.0565994
6322353,2296.0,37.992729,23.731518,0.9026,2020-08-04 08:13:29.119999886,Oktovriou,SB,95.2702,4.24372,-0.0565994
6322354,2296.0,37.992728,23.731518,0.8788,2020-08-04 08:13:29.160000086,Oktovriou,SB,95.1626,4.21791,-0.0565994
